In [1]:
from subprocess import Popen,PIPE

stack='s3://mousebraindata-open/MD657/'

In [2]:
import re
from os import system
import socket

def get_file_table(path):
    system("aws s3 ls %s > awsfiles.txt"%path)
    pat=re.compile(r'(.*lossless)(.*)$')

    T={}
    with open('awsfiles.txt','r') as files:
        for file in files.readlines():
            parts=file.strip().split()
            if len(parts)!=4:
                continue
            filename=parts[3]
            if not 'lossless' in filename:
                continue
            m=pat.match(filename)
            if m:
                file,ext= m.groups()
                info=(ext,parts[1:3])
                if file in T:
                    T[file].append(info)
                else:
                    T[file]=[info]
            else:
                print(filename,'no match')
    return T


In [3]:
T=get_file_table(stack)
!grep patches awsfiles.txt |wc

     464    1854   43971


In [4]:
!grep MD657-F3-2017.02.17-18.07.23_MD657_1_0007_lossless_patches.tgz awsfiles.txt

2018-09-14 23:49:03  160281922 MD657-F3-2017.02.17-18.07.23_MD657_1_0007_lossless_patches.tgz


In [7]:
!grep .lock-ip awsfiles.txt |head -5

2018-09-11 22:48:19         71 MD657-F1-2017.02.17-17.39.26_MD657_1_0001_lossless.lock-ip-172-30-1-84
2018-09-12 22:21:08         71 MD657-F1-2017.02.17-17.39.26_MD657_2_0002_lossless.lock-ip-172-30-1-84
2018-09-15 00:03:47         71 MD657-F1-2017.02.17-17.39.26_MD657_3_0003_lossless.lock-ip-172-30-1-39
2018-09-15 11:28:05         71 MD657-F10-2017.02.17-19.35.42_MD657_1_0028_lossless.lock-ip-172-30-1-4
2018-09-15 09:34:59         72 MD657-F10-2017.02.17-19.35.42_MD657_3_0030_lossless.lock-ip-172-30-1-66


In [8]:
!grep MD657-F88-2017.02.23-17.31.36_MD657_2_0287_lossless awsfiles.txt

In [9]:
T=get_file_table(stack)
!grep patches awsfiles.txt | wc
!grep less.jp2 awsfiles.txt | wc

     464    1854   43971
     463    1852   40227


In [10]:
!grep patches awsfiles.txt |wc

     464    1854   43971


In [13]:
from collections import Counter
T=get_file_table(stack)
C=Counter()
for item in T.items():
    extensions=[x for x in item[1]]
    for ext in extensions:
        if 'lock' in ext[0]:
            C[ext[0]]+=1
C

Counter({'.lock-ip-172-30-1-84': 88,
         '.lock-ip-172-30-1-39': 38,
         '.lock-ip-172-30-1-4': 38,
         '.lock-ip-172-30-1-66': 40,
         '.lock-ip-172-30-1-38': 33,
         '.lock-ip-172-30-1-44': 37,
         '.lock-ip-172-30-1-156': 36,
         '.lock-ip-172-30-1-126': 37,
         '.lock-ip-172-30-1-211': 41,
         '.lock-ip-172-30-1-183': 38,
         '.lock-ip-172-30-1-150': 36})

In [14]:
def run(command):
    print('system cmd=',command)
    system(command)

i=0
for item in T.items():
    #print(item[0])
    extensions=[x for x in item[1]]
    has_lock=False
    is_done=False
    is_small=False
    for ext in extensions:
        #print('++',ext)
        if ('patches'in ext[0]):
            if (int(ext[1][1])>1000):
                is_done=True
            else:
                is_small=True
                patch_name=item[0]+ext[0]
        
        if ('lock' in ext[0]):
            has_lock=True
            lock_name=item[0]+ext[0]

    if has_lock: # and not is_done:
        print('>>>>>>>>>>>>>>>>>>>>>>>> delete lock',lock_name)
        run('aws s3 rm %s%s'%(stack,lock_name))
    if is_small:
        print('====================== delete small patch',patch_name)
        run('aws s3 rm %s%s'%(stack,patch_name))
        
    #if i>10:
    #    break
    i+=1

>>>>>>>>>>>>>>>>>>>>>>>> delete lock MD657-F1-2017.02.17-17.39.26_MD657_1_0001_lossless.lock-ip-172-30-1-84
system cmd= aws s3 rm s3://mousebraindata-open/MD657/MD657-F1-2017.02.17-17.39.26_MD657_1_0001_lossless.lock-ip-172-30-1-84
>>>>>>>>>>>>>>>>>>>>>>>> delete lock MD657-F1-2017.02.17-17.39.26_MD657_2_0002_lossless.lock-ip-172-30-1-84
system cmd= aws s3 rm s3://mousebraindata-open/MD657/MD657-F1-2017.02.17-17.39.26_MD657_2_0002_lossless.lock-ip-172-30-1-84
>>>>>>>>>>>>>>>>>>>>>>>> delete lock MD657-F1-2017.02.17-17.39.26_MD657_3_0003_lossless.lock-ip-172-30-1-39
system cmd= aws s3 rm s3://mousebraindata-open/MD657/MD657-F1-2017.02.17-17.39.26_MD657_3_0003_lossless.lock-ip-172-30-1-39
>>>>>>>>>>>>>>>>>>>>>>>> delete lock MD657-F10-2017.02.17-19.35.42_MD657_1_0028_lossless.lock-ip-172-30-1-4
system cmd= aws s3 rm s3://mousebraindata-open/MD657/MD657-F10-2017.02.17-19.35.42_MD657_1_0028_lossless.lock-ip-172-30-1-4
>>>>>>>>>>>>>>>>>>>>>>>> delete lock MD657-F10-2017.02.17-19.35.42_MD657

>>>>>>>>>>>>>>>>>>>>>>>> delete lock MD657-F19-2017.02.23-16.16.58_MD657_3_0057_lossless.lock-ip-172-30-1-66
system cmd= aws s3 rm s3://mousebraindata-open/MD657/MD657-F19-2017.02.23-16.16.58_MD657_3_0057_lossless.lock-ip-172-30-1-66
>>>>>>>>>>>>>>>>>>>>>>>> delete lock MD657-F2-2017.02.17-17.52.35_MD657_1_0004_lossless.lock-ip-172-30-1-211
system cmd= aws s3 rm s3://mousebraindata-open/MD657/MD657-F2-2017.02.17-17.52.35_MD657_1_0004_lossless.lock-ip-172-30-1-211
>>>>>>>>>>>>>>>>>>>>>>>> delete lock MD657-F2-2017.02.17-17.52.35_MD657_2_0005_lossless.lock-ip-172-30-1-150
system cmd= aws s3 rm s3://mousebraindata-open/MD657/MD657-F2-2017.02.17-17.52.35_MD657_2_0005_lossless.lock-ip-172-30-1-150
>>>>>>>>>>>>>>>>>>>>>>>> delete lock MD657-F2-2017.02.17-17.52.35_MD657_3_0006_lossless.lock-ip-172-30-1-84
system cmd= aws s3 rm s3://mousebraindata-open/MD657/MD657-F2-2017.02.17-17.52.35_MD657_3_0006_lossless.lock-ip-172-30-1-84
>>>>>>>>>>>>>>>>>>>>>>>> delete lock MD657-F20-2017.02.17-22.23.13

>>>>>>>>>>>>>>>>>>>>>>>> delete lock MD657-F34-2017.02.18-02.34.03_MD657_1_0100_lossless.lock-ip-172-30-1-183
system cmd= aws s3 rm s3://mousebraindata-open/MD657/MD657-F34-2017.02.18-02.34.03_MD657_1_0100_lossless.lock-ip-172-30-1-183
>>>>>>>>>>>>>>>>>>>>>>>> delete lock MD657-F34-2017.02.18-02.34.03_MD657_2_0101_lossless.lock-ip-172-30-1-150
system cmd= aws s3 rm s3://mousebraindata-open/MD657/MD657-F34-2017.02.18-02.34.03_MD657_2_0101_lossless.lock-ip-172-30-1-150
>>>>>>>>>>>>>>>>>>>>>>>> delete lock MD657-F35-2017.02.18-02.55.49_MD657_1_0103_lossless.lock-ip-172-30-1-66
system cmd= aws s3 rm s3://mousebraindata-open/MD657/MD657-F35-2017.02.18-02.55.49_MD657_1_0103_lossless.lock-ip-172-30-1-66
>>>>>>>>>>>>>>>>>>>>>>>> delete lock MD657-F35-2017.02.18-02.55.49_MD657_2_0104_lossless.lock-ip-172-30-1-38
system cmd= aws s3 rm s3://mousebraindata-open/MD657/MD657-F35-2017.02.18-02.55.49_MD657_2_0104_lossless.lock-ip-172-30-1-38
>>>>>>>>>>>>>>>>>>>>>>>> delete lock MD657-F36-2017.02.18-03

>>>>>>>>>>>>>>>>>>>>>>>> delete lock MD657-F5-2017.02.17-18.29.04_MD657_1_0013_lossless.lock-ip-172-30-1-44
system cmd= aws s3 rm s3://mousebraindata-open/MD657/MD657-F5-2017.02.17-18.29.04_MD657_1_0013_lossless.lock-ip-172-30-1-44
>>>>>>>>>>>>>>>>>>>>>>>> delete lock MD657-F5-2017.02.17-18.29.04_MD657_2_0014_lossless.lock-ip-172-30-1-38
system cmd= aws s3 rm s3://mousebraindata-open/MD657/MD657-F5-2017.02.17-18.29.04_MD657_2_0014_lossless.lock-ip-172-30-1-38
>>>>>>>>>>>>>>>>>>>>>>>> delete lock MD657-F5-2017.02.17-18.29.04_MD657_3_0015_lossless.lock-ip-172-30-1-183
system cmd= aws s3 rm s3://mousebraindata-open/MD657/MD657-F5-2017.02.17-18.29.04_MD657_3_0015_lossless.lock-ip-172-30-1-183
>>>>>>>>>>>>>>>>>>>>>>>> delete lock MD657-F50-2017.02.18-08.23.22_MD657_1_0148_lossless.lock-ip-172-30-1-84
system cmd= aws s3 rm s3://mousebraindata-open/MD657/MD657-F50-2017.02.18-08.23.22_MD657_1_0148_lossless.lock-ip-172-30-1-84
>>>>>>>>>>>>>>>>>>>>>>>> delete lock MD657-F50-2017.02.18-08.23.22_M

>>>>>>>>>>>>>>>>>>>>>>>> delete lock MD657-F65-2017.02.18-13.26.50_MD657_1_0193_lossless.lock-ip-172-30-1-150
system cmd= aws s3 rm s3://mousebraindata-open/MD657/MD657-F65-2017.02.18-13.26.50_MD657_1_0193_lossless.lock-ip-172-30-1-150
>>>>>>>>>>>>>>>>>>>>>>>> delete lock MD657-F65-2017.02.18-13.26.50_MD657_2_0194_lossless.lock-ip-172-30-1-44
system cmd= aws s3 rm s3://mousebraindata-open/MD657/MD657-F65-2017.02.18-13.26.50_MD657_2_0194_lossless.lock-ip-172-30-1-44
>>>>>>>>>>>>>>>>>>>>>>>> delete lock MD657-F66-2017.02.18-13.46.29_MD657_1_0196_lossless.lock-ip-172-30-1-183
system cmd= aws s3 rm s3://mousebraindata-open/MD657/MD657-F66-2017.02.18-13.46.29_MD657_1_0196_lossless.lock-ip-172-30-1-183
>>>>>>>>>>>>>>>>>>>>>>>> delete lock MD657-F66-2017.02.18-13.46.29_MD657_2_0197_lossless.lock-ip-172-30-1-38
system cmd= aws s3 rm s3://mousebraindata-open/MD657/MD657-F66-2017.02.18-13.46.29_MD657_2_0197_lossless.lock-ip-172-30-1-38
>>>>>>>>>>>>>>>>>>>>>>>> delete lock MD657-F67-2017.02.18-14

...

>>>>>>>>>>>>>>>>>>>>>>>> delete lock MD657-N39-2017.02.22-16.04.05_MD657_2_0116_lossless.lock-ip-172-30-1-66
system cmd= aws s3 rm s3://mousebraindata-open/MD657/MD657-N39-2017.02.22-16.04.05_MD657_2_0116_lossless.lock-ip-172-30-1-66
>>>>>>>>>>>>>>>>>>>>>>>> delete lock MD657-N4-2017.02.27-15.56.07_MD657_1_0010_lossless.lock-ip-172-30-1-38
system cmd= aws s3 rm s3://mousebraindata-open/MD657/MD657-N4-2017.02.27-15.56.07_MD657_1_0010_lossless.lock-ip-172-30-1-38
>>>>>>>>>>>>>>>>>>>>>>>> delete lock MD657-N4-2017.02.27-15.56.07_MD657_2_0011_lossless.lock-ip-172-30-1-211
system cmd= aws s3 rm s3://mousebraindata-open/MD657/MD657-N4-2017.02.27-15.56.07_MD657_2_0011_lossless.lock-ip-172-30-1-211
>>>>>>>>>>>>>>>>>>>>>>>> delete lock MD657-N4-2017.02.27-15.56.07_MD657_3_0012_lossless.lock-ip-172-30-1-84
system cmd= aws s3 rm s3://mousebraindata-open/MD657/MD657-N4-2017.02.27-15.56.07_MD657_3_0012_lossless.lock-ip-172-30-1-84
>>>>>>>>>>>>>>>>>>>>>>>> delete lock MD657-N40-2017.02.22-16.08.37_M

>>>>>>>>>>>>>>>>>>>>>>>> delete lock MD657-N54-2017.02.22-17.07.22_MD657_2_0161_lossless.lock-ip-172-30-1-126
system cmd= aws s3 rm s3://mousebraindata-open/MD657/MD657-N54-2017.02.22-17.07.22_MD657_2_0161_lossless.lock-ip-172-30-1-126
>>>>>>>>>>>>>>>>>>>>>>>> delete lock MD657-N55-2017.02.22-17.12.27_MD657_1_0163_lossless.lock-ip-172-30-1-150
system cmd= aws s3 rm s3://mousebraindata-open/MD657/MD657-N55-2017.02.22-17.12.27_MD657_1_0163_lossless.lock-ip-172-30-1-150
>>>>>>>>>>>>>>>>>>>>>>>> delete lock MD657-N55-2017.02.22-17.12.27_MD657_2_0164_lossless.lock-ip-172-30-1-44
system cmd= aws s3 rm s3://mousebraindata-open/MD657/MD657-N55-2017.02.22-17.12.27_MD657_2_0164_lossless.lock-ip-172-30-1-44
>>>>>>>>>>>>>>>>>>>>>>>> delete lock MD657-N56-2017.02.22-17.16.39_MD657_1_0166_lossless.lock-ip-172-30-1-156
system cmd= aws s3 rm s3://mousebraindata-open/MD657/MD657-N56-2017.02.22-17.16.39_MD657_1_0166_lossless.lock-ip-172-30-1-156
>>>>>>>>>>>>>>>>>>>>>>>> delete lock MD657-N56-2017.02.22-

>>>>>>>>>>>>>>>>>>>>>>>> delete lock MD657-N7-2017.02.27-16.00.18_MD657_2_0020_lossless.lock-ip-172-30-1-84
system cmd= aws s3 rm s3://mousebraindata-open/MD657/MD657-N7-2017.02.27-16.00.18_MD657_2_0020_lossless.lock-ip-172-30-1-84
>>>>>>>>>>>>>>>>>>>>>>>> delete lock MD657-N7-2017.02.27-16.00.18_MD657_3_0021_lossless.lock-ip-172-30-1-211
system cmd= aws s3 rm s3://mousebraindata-open/MD657/MD657-N7-2017.02.27-16.00.18_MD657_3_0021_lossless.lock-ip-172-30-1-211
>>>>>>>>>>>>>>>>>>>>>>>> delete lock MD657-N70-2017.02.22-18.31.14_MD657_1_0208_lossless.lock-ip-172-30-1-66
system cmd= aws s3 rm s3://mousebraindata-open/MD657/MD657-N70-2017.02.22-18.31.14_MD657_1_0208_lossless.lock-ip-172-30-1-66
>>>>>>>>>>>>>>>>>>>>>>>> delete lock MD657-N70-2017.02.22-18.31.14_MD657_2_0209_lossless.lock-ip-172-30-1-84
system cmd= aws s3 rm s3://mousebraindata-open/MD657/MD657-N70-2017.02.22-18.31.14_MD657_2_0209_lossless.lock-ip-172-30-1-84
>>>>>>>>>>>>>>>>>>>>>>>> delete lock MD657-N71-2017.02.22-18.34.41

>>>>>>>>>>>>>>>>>>>>>>>> delete lock MD657-N84-2017.02.22-19.16.42_MD657_3_0252_lossless.lock-ip-172-30-1-4
system cmd= aws s3 rm s3://mousebraindata-open/MD657/MD657-N84-2017.02.22-19.16.42_MD657_3_0252_lossless.lock-ip-172-30-1-4
>>>>>>>>>>>>>>>>>>>>>>>> delete lock MD657-N85-2017.02.22-19.20.56_MD657_1_0253_lossless.lock-ip-172-30-1-39
system cmd= aws s3 rm s3://mousebraindata-open/MD657/MD657-N85-2017.02.22-19.20.56_MD657_1_0253_lossless.lock-ip-172-30-1-39
>>>>>>>>>>>>>>>>>>>>>>>> delete lock MD657-N85-2017.02.22-19.20.56_MD657_2_0254_lossless.lock-ip-172-30-1-44
system cmd= aws s3 rm s3://mousebraindata-open/MD657/MD657-N85-2017.02.22-19.20.56_MD657_2_0254_lossless.lock-ip-172-30-1-44
>>>>>>>>>>>>>>>>>>>>>>>> delete lock MD657-N85-2017.02.22-19.20.56_MD657_3_0255_lossless.lock-ip-172-30-1-4
system cmd= aws s3 rm s3://mousebraindata-open/MD657/MD657-N85-2017.02.22-19.20.56_MD657_3_0255_lossless.lock-ip-172-30-1-4
>>>>>>>>>>>>>>>>>>>>>>>> delete lock MD657-N86-2017.02.22-19.25.03_M

>>>>>>>>>>>>>>>>>>>>>>>> delete lock MD657-N95-2017.02.22-20.25.11_MD657_2_0284_lossless.lock-ip-172-30-1-150
system cmd= aws s3 rm s3://mousebraindata-open/MD657/MD657-N95-2017.02.22-20.25.11_MD657_2_0284_lossless.lock-ip-172-30-1-150
>>>>>>>>>>>>>>>>>>>>>>>> delete lock MD657-N95-2017.02.22-20.25.11_MD657_3_0285_lossless.lock-ip-172-30-1-44
system cmd= aws s3 rm s3://mousebraindata-open/MD657/MD657-N95-2017.02.22-20.25.11_MD657_3_0285_lossless.lock-ip-172-30-1-44
>>>>>>>>>>>>>>>>>>>>>>>> delete lock MD657-N96-2017.02.22-20.28.14_MD657_1_0286_lossless.lock-ip-172-30-1-66
system cmd= aws s3 rm s3://mousebraindata-open/MD657/MD657-N96-2017.02.22-20.28.14_MD657_1_0286_lossless.lock-ip-172-30-1-66
>>>>>>>>>>>>>>>>>>>>>>>> delete lock MD657-N96-2017.02.22-20.28.14_MD657_2_0287_lossless.lock-ip-172-30-1-150
system cmd= aws s3 rm s3://mousebraindata-open/MD657/MD657-N96-2017.02.22-20.28.14_MD657_2_0287_lossless.lock-ip-172-30-1-150
>>>>>>>>>>>>>>>>>>>>>>>> delete lock MD657-N96-2017.02.22-20